In [1]:
!git clone https://github.com/ipavlopoulos/toxic_spans.git

Cloning into 'toxic_spans'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 298 (delta 8), reused 17 (delta 2), pack-reused 268
Receiving objects: 100% (298/298), 1.32 MiB | 7.68 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.3MB/s 
     |████████████████████████████████| 901kB 50.6MB/s 
     |████████████████████████████████| 3.3MB 59.1MB/s 


In [3]:
!nvidia-smi

Sun Apr 25 09:13:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from ast import literal_eval
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
import torch.nn as nn
from tqdm import tqdm
import math
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'   

In [5]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)
logger = logging.getLogger(__name__)
logging.getLogger('transformers').setLevel(logging.ERROR)

In [6]:
translationTable = str.maketrans("éàèùaêóïü","eaeuaeoiu")

In [7]:
tsd = pd.read_csv("toxic_spans/data/tsd_train.csv") 
tsd.text = tsd.text.apply(lambda x:x.translate(translationTable))
tsd.spans = tsd.spans.apply(literal_eval)
tsd.tail(5)

,spans,text
7934,"[8, 9, 10, 11]",Another fool pipes in.
7935,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...",So if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11]",This idiotic. Use the surplus to pay down the ...
7938,"[106, 107, 108, 109, 110, 169, 170, 171, 172, ...","Who is this ""we"" of which you speak? Are you r..."


In [8]:
text_list = tsd.text.to_list()
spans_list = tsd.spans.to_list()

In [9]:
print(len(text_list))
print(len(spans_list))

7939
7939


In [10]:
model_name = "google/electra-large-discriminator"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

04/25/2021 09:13:24 - Lock 140159506594832 acquired on /root/.cache/huggingface/transformers/344f5be314c0b91e28096c6730a1a43d61ba11aee91fd8ff026aba39138181d1.42a26b08340748782d961639bc9f49bce229c0f3a1e6c3f0142221e387d80169.lock


04/25/2021 09:13:24 - Lock 140159506594832 released on /root/.cache/huggingface/transformers/344f5be314c0b91e28096c6730a1a43d61ba11aee91fd8ff026aba39138181d1.42a26b08340748782d961639bc9f49bce229c0f3a1e6c3f0142221e387d80169.lock


04/25/2021 09:13:25 - Lock 140161019302800 acquired on /root/.cache/huggingface/transformers/48a62a60c85c63546f3623e592c2ddfd0628ed7749e6d503a11eb80cb04fc19c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


04/25/2021 09:13:25 - Lock 140161019302800 released on /root/.cache/huggingface/transformers/48a62a60c85c63546f3623e592c2ddfd0628ed7749e6d503a11eb80cb04fc19c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


04/25/2021 09:13:25 - Lock 140159654409808 acquired on /root/.cache/huggingface/transformers/ed8095412008e8a8159d4bbdcecd02e5e72b79a1fc7dbfdc32e6aef638d4b9a9.65c74b3f0086fae55b99a8c9173a6739a53ae5ae0441c0811095141532f33ff8.lock


04/25/2021 09:13:26 - Lock 140159654409808 released on /root/.cache/huggingface/transformers/ed8095412008e8a8159d4bbdcecd02e5e72b79a1fc7dbfdc32e6aef638d4b9a9.65c74b3f0086fae55b99a8c9173a6739a53ae5ae0441c0811095141532f33ff8.lock


04/25/2021 09:13:26 - Lock 140159503397520 acquired on /root/.cache/huggingface/transformers/26ad81c46898598ce9aed0b02fd3c9175a28daa30317e4f1980b5e871d823b67.4f2213f5603276adf12967b32e4444c0f187f34ca4f8b22a65f03e13514589e9.lock


04/25/2021 09:13:27 - Lock 140159503397520 released on /root/.cache/huggingface/transformers/26ad81c46898598ce9aed0b02fd3c9175a28daa30317e4f1980b5e871d823b67.4f2213f5603276adf12967b32e4444c0f187f34ca4f8b22a65f03e13514589e9.lock


In [12]:
special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}

In [13]:
def find_idx(sentence, token, position=0):
    start = sentence.find(token, position)
    end = start + len(token)
    if start == -1:
        return []
    return list(range(start,end))

In [14]:
def encode_and_trans_labels(text_list, spans_list):
    inputs = tokenizer(
        text_list,                      
        add_special_tokens = True,             
        truncation=True,
        padding = 'max_length',     
        return_tensors = 'pt',
        max_length = 256
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    labels = []
    for ids, sentence, toxic_spans in zip(input_ids, text_list, spans_list):
        # print(sentence)
        # print(toxic_spans)
        tokens = tokenizer.convert_ids_to_tokens(ids)
        token_labels = []
        position = 0
        for token in tokens[:]:
            if token in special_tokens.values():
                token_labels.append(0.)
            else:
                token = token.replace("##","")
                spans = find_idx(sentence.lower(), token, position=position)
                if spans == []:
                    spans = list(range(position,position+len(token)))
                    print("not find:",token,spans,position,[sentence[i] for i in spans])
                # print(token,spans,position)
                position = spans[-1]+1
                if set(spans[:]) <= set(toxic_spans) or (set(toxic_spans)<=set(spans) and len(set(toxic_spans))>0):
                    token_labels.append(1.)
                elif len(set(spans[:-1])) > 0 and (set(spans[:-1]) <= set(toxic_spans)):
                    token_labels.append(1.)
                else:
                    token_labels.append(0.)
                # print(token,spans,position)
        labels.append(token_labels)
    labels = torch.tensor(labels)

    return input_ids, attention_mask, labels

In [15]:
class TextDataSet(Dataset):
    def __init__(self, input_ids, attention_mask, labels, text_list, spans_list):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.text_list = text_list
        self.spans_list = spans_list

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):   
        return (self.input_ids[idx], self.attention_mask[idx], self.labels[idx], self.text_list[idx], self.spans_list[idx])

In [16]:
input_ids,attention_mask,labels = encode_and_trans_labels(text_list, spans_list)

In [22]:
dataset = TextDataSet(input_ids, attention_mask, labels, text_list, spans_list)
train_size = int(len(dataset)*0.8)
valid_size = len(dataset)-train_size
train_dataset, valid_dataset = random_split(dataset, [train_size,valid_size])
print('Train samples: {}, Valid samples: {}'.format(len(train_dataset), len(valid_dataset)))

Train samples: 6351, Valid samples: 1588


In [23]:
import re
import collections
from torch._six import string_classes

np_str_obj_array_pattern = re.compile(r'[SaUO]')

def default_collate(batch):
    r"""Puts each data field into a tensor with outer dimension batch size"""

    elem = batch[0]
    elem_type = type(elem)
    if isinstance(elem, torch.Tensor):
        out = None
        if torch.utils.data.get_worker_info() is not None:
            # If we're in a background process, concatenate directly into a
            # shared memory tensor to avoid an extra copy
            numel = sum([x.numel() for x in batch])
            storage = elem.storage()._new_shared(numel)
            out = elem.new(storage)
        return torch.stack(batch, 0, out=out)
    elif elem_type.__module__ == 'numpy' and elem_type.__name__ != 'str_' \
            and elem_type.__name__ != 'string_':
        if elem_type.__name__ == 'ndarray' or elem_type.__name__ == 'memmap':
            # array of string classes and object
            if np_str_obj_array_pattern.search(elem.dtype.str) is not None:
                raise TypeError(default_collate_err_msg_format.format(elem.dtype))

            return default_collate([torch.as_tensor(b) for b in batch])
        elif elem.shape == ():  # scalars
            return torch.as_tensor(batch)
    elif isinstance(elem, float):
        return torch.tensor(batch, dtype=torch.float64)
    elif isinstance(elem, int):
        return torch.tensor(batch)
    elif isinstance(elem, string_classes):
        return batch
    elif isinstance(elem, list):
        return batch
    elif isinstance(elem, collections.abc.Mapping):
        return {key: default_collate([d[key] for d in batch]) for key in elem}
    elif isinstance(elem, tuple) and hasattr(elem, '_fields'):  # namedtuple
        return elem_type(*(default_collate(samples) for samples in zip(*batch)))
    elif isinstance(elem, collections.abc.Sequence):
        # check to make sure that the elements in batch have consistent size
        it = iter(batch)
        elem_size = len(next(it))
        if not all(len(elem) == elem_size for elem in it):
            raise RuntimeError('each element in list of batch should be of equal size')
        transposed = zip(*batch)
        return [default_collate(samples) for samples in transposed]

    raise TypeError(default_collate_err_msg_format.format(elem_type))

In [24]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_collate)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, collate_fn=default_collate)

In [25]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)

04/25/2021 09:13:33 - Lock 140159506224976 acquired on /root/.cache/huggingface/transformers/6a9790a4cce0d5f0f7d5c78b57955d681fe9cb564edc75aab3733c5ba3a5550d.a2ee8c7426aca3bd41c92ad0b3e07d731d9bf61c950403e6a82b1d566b8923db.lock


04/25/2021 09:13:55 - Lock 140159506224976 released on /root/.cache/huggingface/transformers/6a9790a4cce0d5f0f7d5c78b57955d681fe9cb564edc75aab3733c5ba3a5550d.a2ee8c7426aca3bd41c92ad0b3e07d731d9bf61c950403e6a82b1d566b8923db.lock


In [26]:
weight_CE = torch.FloatTensor([1,2]).to(device)
loss_fct = nn.CrossEntropyLoss(weight=weight_CE)
def loss_fn(logits, attention_mask, labels):
    active_loss = attention_mask.view(-1) == 1
    active_logits = logits.view(-1, 2)
    active_labels = torch.where(
        active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
    )
    loss = loss_fct(active_logits, active_labels)
    return loss

In [27]:
class Trainer:

    def __init__(self, model, train_loader, valid_loader, config):
        self.model = model
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        os.makedirs(self.config.ckpt_path, exist_ok=True)
        logger.info("Save model to {}".format(self.config.ckpt_path))
        torch.save(raw_model.state_dict(), self.config.ckpt_path+"best_model.pt")

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.learning_rate, betas=config.betas)
        def run_epoch(split):
            is_train = (split == 'train')
            model.train(is_train)
            loader = self.train_loader if is_train else self.valid_loader
            losses = []
            spans_list_all = []
            spans_pred_all = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (input_ids, attention_mask, y, text_list, spans_list) in pbar:
                # place data on the correct device
                input_ids = input_ids.to(self.device)
                attention_mask = attention_mask.to(self.device)
                y = y.to(self.device).long()
                # forward the model
                with torch.set_grad_enabled(is_train):
                    outputs = model(input_ids, attention_mask, labels=y)
                    logits = outputs.logits
                    # loss = outputs.loss
                    loss = loss_fn(logits, attention_mask, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())

                    spans_pred = decode_and_trans_labels(text_list, input_ids, logits)
                    # print("pred: ",spans_pred)
                    # print("gold: ",spans_list)
                    f1_score, recall_score, precision_score = batch_score(spans_pred, spans_list)

                    spans_list_all.extend(spans_list)
                    spans_pred_all.extend(spans_pred)

                    # gold = ("".join(text_list[0][i] for i in spans_list[0])).lower()
                    # spans_labels = torch.nonzero(y[0].cpu().detach())
                    # spans_labels = tokenizer.convert_ids_to_tokens([input_ids[0][i] for i in spans_labels])
                    # spans_labels = "".join([i.replace("##","") for i in spans_labels])
                    # pred = "".join(text_list[0][i] for i in spans_pred[0])
                    # print("\ngold: {}\nlables: {}\npred: {}".format(gold,spans_labels,pred))
                
                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += batch_size # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description("epoch {} iter {}: train loss {:.5f}, f1 {:.2f}%, recall {:.2f}%, precision {:.2f}%, lr {:e}"\
                                         .format(epoch+1,it,loss.item(),f1_score*100,recall_score*100,precision_score*100,lr))
                    # pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}, f1 {f1_score:.2f}, recall_score {recall_score:.5f}, lr {lr:e}")

            if not is_train:
                valid_loss = float(np.mean(losses))
                valid_f1_score, valid_recall_score, valid_precision_score = batch_score(spans_pred_all, spans_list_all)
                logger.info("valid loss: {:.5f}".format(valid_loss))
                logger.info("valid f1 score: {:.2f}%".format(valid_f1_score*100))
                logger.info("valid recall: {:.2f}%".format(valid_recall_score*100))
                logger.info("valid precision: {:.2f}%".format(valid_precision_score*100))
                return valid_loss

        self.tokens = 0 # counter used for learning rate decay
        best_loss = float('inf')
        # valid_loss = run_epoch('valid')
        for epoch in range(config.max_epochs):
            
            run_epoch('train')
            if self.valid_loader is not None:
                valid_loss = run_epoch('valid')
            # supports early stopping based on the valid loss, or just save always if no valid set is provided
            good_model = self.valid_loader is None or valid_loss < best_loss
            if self.config.ckpt_path is not None and good_model:
                best_loss = valid_loss
                self.save_checkpoint()

In [28]:
def decode_and_trans_labels(text_list, input_ids, logits):
    token_labels = torch.argmax(logits, dim=-1).cpu().detach().numpy()
    spans_pred = []
    for ids, sentence, labels in zip(input_ids, text_list, token_labels):
        tokens = tokenizer.convert_ids_to_tokens(ids)
        position = 0
        toxic_spans = []
        token2span_list = []
        # print(sentence)
        for token, label in zip(tokens, labels):
            if token in special_tokens.values():
                continue
            token = token.replace("##","")
            spans = find_idx(sentence.lower(), token, position=position)
            # print(token,spans)
            if spans == []:
                spans = list(range(position,position+len(token)))
                print("not find:",token,spans,position,"".join([sentence[i] for i in spans]))
            position = spans[-1]+1
            if label == 1:
                toxic_spans.extend(spans)
                token2span = ("".join([sentence[i] for i in spans])).lower()
                if token != token2span:
                    print("token: ",token)
                    print("token2span: ",token2span)
                token2span_list.append(token2span)
        spans_pred.append(toxic_spans)
        # print(" ".join(token2span_list))
    return spans_pred

In [29]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [30]:
def recall(predictions, gold):
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = len(predictions_set.intersection(gold_set))
    denom = len(gold_set)
    return float(nom)/float(denom)

def precision(predictions, gold):
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = len(predictions_set.intersection(gold_set))
    denom = len(predictions_set)
    return float(nom)/float(denom)

In [31]:
def batch_score(spans_pred, spans_list):
    f1_scores = []
    recall_scores = []
    precision_scores = []
    for pred, gold in zip(spans_pred, spans_list):
        f1_score = f1(pred, gold)
        recall_score = recall(pred, gold)
        precision_score = precision(pred, gold)
        f1_scores.append(f1_score)
        recall_scores.append(recall_score)
        precision_scores.append(precision_score)
    return np.mean(f1_scores), np.mean(recall_scores), np.mean(precision_scores)

In [32]:
class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    learning_rate = 1e-5
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # may useful optimize method
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False # optimize method
    warmup_tokens = 375e6 # use this to train model from a lower learning rate
    final_tokens = 260e9 # all tokens during whole training process
    # checkpoint settings
    ckpt_path = './models/' # save model path

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            print(k,v)
            setattr(self, k, v)

In [33]:
# print model all parameters and parameters need training
print('{} : all params: {:4f}M'.format(model._get_name(), sum(p.numel() for p in model.parameters()) / 1000 / 1000))

ElectraForTokenClassification : all params: 334.094338M


In [34]:
max_epochs = 6
final_tokens = max_epochs * batch_size * len(train_loader)
warmup_tokens = final_tokens//10
tconf = TrainerConfig(max_epochs=max_epochs, learning_rate=1e-5, lr_decay=True, 
                      warmup_tokens=warmup_tokens, final_tokens=final_tokens)

max_epochs 6
learning_rate 1e-05
lr_decay True
warmup_tokens 3811
final_tokens 38112


In [35]:
trainer = Trainer(model, train_loader, valid_loader, tconf)

In [36]:
trainer.train()

epoch 1 iter 793: train loss 0.41167, f1 67.28%, recall 72.14%, precision 77.78%, lr 9.865205e-06: 100%|██████████| 794/794 [10:31<00:00,  1.26it/s]
04/25/2021 09:25:36 - valid loss: 0.26331
04/25/2021 09:25:36 - valid f1 score: 67.58%
04/25/2021 09:25:36 - valid recall: 68.97%
04/25/2021 09:25:36 - valid precision: 76.81%
04/25/2021 09:25:36 - Save model to ./models/
epoch 2 iter 793: train loss 0.34693, f1 68.64%, recall 77.58%, precision 83.75%, lr 8.431159e-06: 100%|██████████| 794/794 [10:31<00:00,  1.26it/s]
04/25/2021 09:37:06 - valid loss: 0.26373
04/25/2021 09:37:06 - valid f1 score: 68.05%
04/25/2021 09:37:06 - valid recall: 71.99%
04/25/2021 09:37:06 - valid precision: 75.32%
epoch 3 iter 793: train loss 0.21808, f1 63.27%, recall 58.39%, precision 71.43%, lr 5.868191e-06: 100%|██████████| 794/794 [10:31<00:00,  1.26it/s]
04/25/2021 09:48:30 - valid loss: 0.27871
04/25/2021 09:48:30 - valid f1 score: 67.57%
04/25/2021 09:48:30 - valid recall: 72.81%
04/25/2021 09:48:30 - val

In [37]:
model.load_state_dict(torch.load('models/best_model.pt'))

<All keys matched successfully>

In [38]:
tsd_test = pd.read_csv("toxic_spans/data/tsd_test.csv") 
tsd_test.text = tsd_test.text.apply(lambda x:x.translate(translationTable))
tsd_test.spans = tsd_test.spans.apply(literal_eval)
tsd_test.tail(5)

,spans,text
1995,"[4, 5, 6, 7, 8, 70, 71, 72, 73, 74, 75, 76, 77...",hey loser change your name to something more a...
1996,"[23, 24, 25, 26, 27]",And you are a complete moron who obviously doe...
1997,"[157, 158, 159, 160, 161, 162, 163, 164, 165, ...",Such vitriol from the left. Who would have th...
1998,[],It is now time for most of you to expand your ...
1999,"[828, 829, 830, 831]","Why does this author think she can demand, or ..."


In [39]:
text_list_test = tsd_test.text.to_list()
spans_list_test = tsd_test.spans.to_list()

In [40]:
print(len(text_list_test))
print(len(spans_list_test))

2000
2000


In [41]:
def predict(text,spans):
    input_ids,attention_mask,labels = encode_and_trans_labels([text], [spans])
    outputs = model(input_ids.to(device), attention_mask.to(device), labels=labels.to(device).long())
    logits = outputs.logits 
    spans_pred = decode_and_trans_labels([text], input_ids, logits)[0]
    # gold = "".join(text[i] for i in spans)
    # pred = "".join(text[i] for i in spans_pred)
    # print("gold: {}\npred: {}".format(gold,pred))
    f1_score = f1(spans_pred, spans)
    recall_score = recall(spans_pred, spans)
    precision_score = precision(spans_pred, spans)
    # print("f1 {:.2f}%, recall {:.2f}%, precision {:.2f}%,".format(f1_score*100,recall_score*100,precision_score*100))
    return spans_pred, f1_score,recall_score,precision_score

In [42]:
f1_scores,recall_scores,precision_scores = [],[],[]
predictions = []
for i in range(len(text_list_test)):
    spans_pred, f1_score,recall_score,precision_score = predict(text_list[i],spans_list[i])
    f1_scores.append(f1_score)
    recall_scores.append(recall_score)
    precision_scores.append(precision_score)
    predictions.append(spans_pred)
print("All: f1 {:.2f}%, recall {:.2f}%, precision {:.2f}%,".format(np.mean(f1_scores)*100,np.mean(recall_scores)*100,np.mean(precision_scores)*100))

All: f1 69.26%, recall 70.17%, precision 79.10%,


In [43]:
# write in a prediction file named "spans-pred.txt"
with open("spans-pred.txt", "w") as out:
    for it, text_scores in enumerate(predictions):
        out.write("{}\t{}\n".format(str(it),str(text_scores)))

! head spans-pred.txt

0	[8, 9, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 51, 52, 53, 54, 55, 56, 57, 58]
1	[33, 34, 35, 36, 37, 38, 39]
2	[0, 1, 2, 3]
3	[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
4	[]
5	[93, 94, 95, 96, 97, 278, 279, 280, 281, 282, 283, 284]
6	[32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44]
7	[0, 1, 2, 3]
8	[49, 50, 51, 52, 53, 54, 147, 148, 149, 150, 151, 152]
9	[0, 1, 2, 3, 4, 5, 6, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39]


In [44]:
# mount my Google Drive directory and access the training data located there
from google.colab import drive
gdrive_dir = '/content/gdrive/'
drive.mount(gdrive_dir, force_remount=True)

Mounted at /content/gdrive/


In [47]:
!cp -r models spans-pred.txt /content/gdrive/MyDrive/ToxicSpans